Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Alejo González García" 
COLLABORATORS = ""

---

# Second graded practice

You will have to handle a notebook with:

* The working programs that solve the exercises. I will run all the codes.
* An explanation of the methods you are using.
* An explanation of your procedure to calculate the solutions.
* An explanation of the results.

## Optimal quadrature nodes and weights

In this exercise we want to optimize the integration of polynomials using quadrature formulas:

$$\int_{-1}^{1} p_n(x) dx = \sum_{i=1}^{M} \omega_i p_n(x_i)$$

Here $p_n(x)$ is a degree $n$ polynomial, $\left\{x_i \right\}$ are $M$ nodes inside the interval $[-1,1]$ and $\omega_i$ are the weights of the quadrature formula.

Let us suppose that you can use only two nodes, but you can choose which ones, and also you are free to choose the corresponding weights.

Optimize the node positions and the value of the weights so your formula is exact for polynomials of the highest possible order.

In order to do tackle this problem you need to code your own optimization method. 

The function you will have to minimize in this problem will not have an analytic expression, but no matter the method you choose, you will have to calculate the gradient. Think about finite differences.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy

**Explanation of the path taken to solve the problem**

The target on this assignment is to solve an optimization problem. I know we were provided many information about how to achieve the function to optimize but I am going to proof I understand it: 

We can manually solve this problem, in the analytic way. We know that with n equations we are going to be capable of integrating polynomials of order n-1, as for this practice we are using only 4 variables (x1, x2, w1, w2), we can integrate without computing error **up to degree $x^3$**. 

Also note that the integral we are computing is between bounds -1 and 1 meaning that if we integrate any polynomial with odd degree, the resulting integral will be zero, so we have to avoid been confused by this fact. 

To find the equation that we have to optimize we have to take each of the equations that we obtain when using the approximations to integrate 1, x, $x^2$ and $x^3$; and compute the error with respect to the real value of the integral. 

The square of each of this substractions is the formula we have to optimize. A very important fact, is that we need to compute the square of each equation, so that we obtain the optimal correct points, otherwise we will optimize the function and get really low error but the points will not work in reality. 

So, if we compute by Trapezoid Rule (will see it below) the integrals and then substract and elevate to the square each of the terms, we will get: 

Error(x1, x2, w1, w2) = $(2 - $w1$ - $w2$)^2$ + $($w1$ * $x1$ + $w2$ * $x2$)^2$ + $($2/3$ - $w1$ * $(x1^2)$ - w2 * $x2^2$)^2$+
$($w1$ * $x1^3$ + $w2$ * $x2^3$)^2$

Once we have the function defined, we need to optimize it, to do so we are going to use the next steps:

  1. Implement **Composite Trapezoid Rule** to compute the Real value of each of the integrals and check our results.
  2. Create a method to **compute the gradient** of a function with 4 variables by using finite differences.
  3. Use a method to optimize the function, I have chosen **Gradient Descent** but any other is fine too.
  
Now that I have explained what is the approach we are going to follow, I am going to show the code followed to solve the practice and explain how I computed each of the steps. 



In [3]:
def compositeTrapezoid(f,a, b, n):

        # f is the function that we want to integrate.
        # a is the minimum value the integral can take
        # b is the maximum possible value
        # n is the number of points we consider
                
        H = (b-a) / (n)
        currentPlusNextValue = 0
        
        for k in range(0, n): 
                currentPlusNextValue +=  f[k] + f[k+1] 
        
        return currentPlusNextValue * (H / 2) 
    
# Composite Trapezoid Rule Explanation: 
    # I have taken this method from the first practice, it allows me to compute the value of any integral with
    # a high precision so that we can take it as the real value. 
    
    # What it does is multiplying the step size times the sum of the values of the function 
    # at time, k, plus the next time value.
    # At the end it gives the value of the integral in the provided bounds. 
    

In [4]:
def computeAListOfIntegrals(lowerBound, upperBound, functionsToIntegrate, n):
    
    solutionsOfTheIntegrals = []

    for i in range(0, len(functionsToIntegrate)):
        f = functionsToIntegrate[i]
        solutionsOfTheIntegrals.append(compositeTrapezoid(f, lowerBound, upperBound, n))   
    return solutionsOfTheIntegrals

# ComputeAListOfIntegrals Explanation:
    # This is a method that I have implemented so that we can compute the integrals of many functions. 
    # It computes each of the integrals with Trapezoid rule (method above) and plugs them in a list.  

In [5]:
def computeGradientInFourVariables(x, y, z, w, f, h):
    
        # f: function from which I am computing the gradient. 
        # h: step size I take. In general the smaller the better, but so small implies so slow. 
        # partial X, Y, Z, W: Partial Derivative with respect to X, Y, Z, W.

    partialX = ((f(x+h, y, z, w) - f(x-h, y, z, w)) / (2*h))
    partialY = ((f(x, y+h, z, w) - f(x, y-h, z, w)) / (2*h))
    partialZ = ((f(x, y, z+h, w) - f(x, y, z-h, w)) / (2*h)) 
    partialW = ((f(x, y, z, w+h) - f(x, y, z, w-h)) / (2*h))

    return np.array([partialX, partialY, partialZ, partialW], dtype=object)

# ComputeGradientInFourVariables Explanation: 
    
    # This method is as simpler as it seems to be. 
    # If we apply finite differences in the multivariate case, the only difference with one variable case is 
    # that we only need to sum and substract the step size on the variable we are computting the derivate at. 
    
    # After computing each of the partial derivatives, we plug them all togheter on an array. 

In [6]:
def ErrorFunction(x1, x2, w1, w2):
    return ((2 - w1 - w2)**2 + (w1*x1 + w2*x2)**2 + ((2/3) - w1*(x1**2) - w2*(x2**2))**2 + (w1*(x1**3) + w2*(x2**3))**2)
    
# ErrorFunction Explanation: 
    # This function is the error function, the one we defined previously in the statement, the one we are going 
    # to optimize. It depends on the 4 possible variables. 
    
    # Remark that values 2 and 2/3 have been computed with Trapezoid Rule but
    # it is an easy integral that takes few time by hand

In [7]:
def gradientDescend(f, initialTimePoints, alpha, tol):
    
    # f is the function we are optimizing.
    # initialTimePoints is the set of 4 initial points from which we start to find the minimum of the function. 
    # alpha is a tricky value, the one that reduces the step size at each iteration and we will deal with. 
    # tol is the stopping criteria, the error we tolerate to have. 
    
    
    t = initialTimePoints.copy() # We are storing the initial points in a variable for further use. 
    
    previousInitTimePoints = initialTimePoints - 10 * tol # Obtain the previous points, to the assigned ones this
                                                          # selection depends a lot on the tolerance we select. 
    
    maxIterationsAllowed = 5000 # Number of maximum iterations I allow to use (We will not use more than 2000).
    iterationCount = 0 # Iteration number at which we are located. 
    initialAlphaProvided = alpha # Storing the initial value of alpha so that I can reset it whenever I need it. 

    resetAlphaWhenAchieved = 3 # I will reset the value of alpha each 3 iterations, so that the step size is not so small. 
                               # If I decrease this number up to 2, depending on the initial points, the number of iterations 
                               # is reduced. 
            
    resetAlphaCounter = 0 # Iteration number (between 0 and resetValue) at which alpha is located. 
    
    while np.linalg.norm(t - previousInitTimePoints) > tol and iterationCount < maxIterationsAllowed:
        # If have not achieved the desired accuracy and the max number of iterations is not achieved, I do not stop.  

        previousInitTimePoints = t.copy()
        t = t -  (alpha * computeGradientInFourVariables(t[0], t[1], t[2], t[3], f, h = 0.00001))
        
        # This is the key point, I substract to the current point t, alpha times the gradient of the function 
        # at points 0, 1, 2 and 3 of my current point. 
        # In this way I keep reducing the distance to the minimum in a descent direction continuosly. 
        
        iterationCount += 1 # Increase iteration count at which we are. 
        alpha =  (1/2) * alpha # Decrease the value of alpha
        resetAlphaCounter += 1 # Increase iteration at which alpha is. 

        print(ErrorFunction(t[0], t[1], t[2], t[3])) # This shows how are we doing progress in each of the iterations. 

        if resetAlphaWhenAchieved == resetAlphaCounter: # When the reset value of alpha is achieved.

            alpha = initialAlphaProvided # Reset alpha to original value.
            resetAlphaCounter = 0 # Set reset of alpha to 0 again.
    
    return t, iterationCount # Return the minimum and the number of iterations we computed. 

# Gradient Descend Method Explained: 
    # I have explained at each step what is the method doing so maybe it seems a bit long. 
    # What we are doing at the end is computing the gradient at the given points and at each step 
    # we make sure about we are decreasing into the good direction and we are getting closer to the minimum. 
    
    # All the alpha related stuff could be removed and the method keeps working if we just set alpha to 1/2, 
    # but I think that reseting the value each N steps is a good improvement for the method. 

In [8]:
def estimation(x, w0, w1, pn):
    return(w0 * pn(x[0]) + w1 * pn(x[1]))

# Estimation Explanation:
    # This method is the approximation that we must fulfill at the end, as said by the statement of the problem.
    # It multiplies each of the weights by the value of the polynomial we want to approximate at the correspoing point x. 

In [9]:
# Now that we have defined all the methods let´s check what is the result of our optimization problem: 

initialPointsGuess = [0.1, 0.8, 2, 1.2] # Set of initial points from which we want to find the minimum of the function. 

# initialPointsGuess = [1, 0.4, 0.3, 0.9] 
# initialPointsGuess = [-1, 0.4, -0.3, 0.9]  # I have randomly chosen this points and the method works for any of them.  
# initialPointsGuess = [-1, -0.4, 0.7, -0.9] 


tol = 10e-15 # Maximum error value that we want to have. 
alpha = 0.2 # Value of alpha that we choose. It should be in the bounds (0-1)

finalPointsEstimation = gradientDescend(ErrorFunction, np.array(initialPointsGuess), alpha, tol)
# This is the result that we obtain by minimizing the proposed function. 

Achieved_iterations = finalPointsEstimation[1] # Number of iterations computed.
GradientDescendSolution = finalPointsEstimation[0] # Value of each of the 4 variables we have optimized. 


3.0661384068393223
0.987601734138661
0.7354015108577316
0.2506290250075848
0.18108071425498445
0.15089697708792887
0.060131234395041894
0.04389967756983592
0.03962575940098741
0.030210913473111348
0.028286739842355427
0.027480077260407347
0.024586779738700482
0.02335063364151406
0.022763255594079036
0.02052225658723232
0.01950052089393893
0.019011719022423615
0.017140588565798778
0.01628493629021195
0.01587556524510398
0.014308659076091673
0.013592406603872297
0.01324981197914303
0.01193880558969185
0.011339808196451945
0.011053359683790454
0.009957426036997018
0.009456891018574609
0.00921757074593034
0.008302097323828611
0.007884117024228688
0.007684297625827331
0.006920030916716249
0.006571180429776706
0.006404429226530608
0.005766712885711153
0.005475690561581524
0.00533659515739212
0.0048046938856182685
0.00456200458135657
0.004446019703206504
0.00400252726571045
0.003800206597196737
0.003703521093600605
0.0033338474212261794
0.003165223888732248
0.0030846462145269083
0.00277657656

In [10]:
# Let´s check what is the Error we are getting. 

RealSolutionProvidedByTrapezoidRule = [-1/np.sqrt(3), 1/np.sqrt(3), 1, 1];
# This is the analitical solution that I have computed manually. 
# If we substitute this values on the error function it should return 0. 

print("Analitical solution substitued on the Error Function: ",
      ErrorFunction(RealSolutionProvidedByTrapezoidRule[0], RealSolutionProvidedByTrapezoidRule[1], 
                    RealSolutionProvidedByTrapezoidRule[2], RealSolutionProvidedByTrapezoidRule[3]))


print("Approximation computed on Gradient Descent substitued on the Error Function:  ", 
      ErrorFunction(GradientDescendSolution[0], GradientDescendSolution[1], 
                    GradientDescendSolution[2], GradientDescendSolution[3]))


print("\nInitial points chosen: ", initialPointsGuess)
print("Final Points Achieved by Gradient: ", GradientDescendSolution)
print("Number of Iterations computed: ", Achieved_iterations)

# We can see that the error that we take with our approximation is quite low, to the power of 20, 
# and the points that we get as solution are also really accurate. 

Analitical solution substitued on the Error Function:  4.930380657631324e-32
Approximation computed on Gradient Descent substitued on the Error Function:   6.94444365664632e-20

Initial points chosen:  [0.1, 0.8, 2, 1.2]
Final Points Achieved by Gradient:  [0.5773502690691473 -0.5773502690695417 1.0000000000421623
 1.0000000000411708]
Number of Iterations computed:  900


In [11]:
# To finish, we need to make sure that effectively by minimizing this error function we are really computing 
# some value for which we can approximate any integral of degree up to 3. Let´s do it: 


n = 1000 # Number of points we want to take. 
lowerBound = -1 # Minimum value of the region at which we integrate.
upperBound = 1 # Maximum value of the region. 
t = np.linspace(lowerBound, upperBound, n+1) # time line 

p = [5*t**3 - 7*t**2 + 6*t - 2] # Function Example to prove we are doing this well. 
pn = lambda x: 5*x**3 - 7*x**2 + 6*x - 2 # The same function but defined with lambda x. 

realValueComputedWithTrapezoidRule = computeAListOfIntegrals(lowerBound, upperBound, p, n) 
# Here I am calling method computeAListOfIntegrals above that makes the integral with Composite Trapezoid Rule
# very accurately. 


print("Real value of the integral: ", realValueComputedWithTrapezoidRule[0])

approximationComputedWithGradient = estimation([GradientDescendSolution[0], GradientDescendSolution[1]], 
                 GradientDescendSolution[2], GradientDescendSolution[3], pn)

print("Value of the estimation of the integral using Gradient: ", approximationComputedWithGradient)

errorCommited = (realValueComputedWithTrapezoidRule[0] - approximationComputedWithGradient)**2

print("The error comitted while computing the integral is: ", errorCommited)

# We have to take into account that the method that computes the "Real" integral has also a small error, so in fact 
# this error is small and maybe even our approximation is better than the method we have implemented. 

Real value of the integral:  -8.666676000000008
Value of the estimation of the integral using Gradient:  -8.666666665083282
The error comitted while computing the integral is:  8.714067027825283e-11


**Final Conclusions**

At the end we can see that we are achieving an error in minimizing the Error function of 10e-20, what is a really good solution and 10e-11 while computing any of the integrals. 

Remark again that what I have done is first obtaining the Composite Trapezoid from the first assignment, then compute the gradient in four variables with finite differences and to finish, minimize the error function with Gradient Descend. 

I have to say that this assignment has been a bit more difficult than the first one, as it is expected if we take into account that we are in a higher dimension (4 variables) so we cannot even plot the function to obtain the minimum, but it has been challenging to compute each of the steps and I have liked it. 

I have computed each of the methods personally, by my self, and the same applies with the analytical solution.

Also I think that by doing this kind of assignments is the way that we really learn what we are doing, and as you do not have to memorize each function and we have the computer, we can make hard computations and learn to solve difficult problems for the future. 